Hi! This shows you exactly how we have made the training model for Mentu, an AI lie detector. We have already trained a model for voice consistency in Google's Teachable Machine. We will then be connecting this to TenserFlow in python in order to render it. First get the required apps for this part of the project. You will find all of them in requirements/base.txt in my github. Next, simply download the teachable machine models using these code snippets.

In [ ]:
import requests

model_url_base = "https://teachablemachine.withgoogle.com/models/goMB9ACbx/"

# Download the model.json file
model_json_url = model_url_base + "model.json"
response_json = requests.get(model_json_url)
with open("model.json", "wb") as f:
    f.write(response_json.content)
print("Downloaded model.json")

# Download the weights.bin file
weights_bin_url = model_url_base + "weights.bin"
response_weights = requests.get(weights_bin_url)
with open("weights.bin", "wb") as f:
    f.write(response_weights.content)
print("Downloaded weights.bin")

# Download the metadata.json file (optional, but useful)
metadata_url = model_url_base + "metadata.json"
response_metadata = requests.get(metadata_url)
with open("metadata.json", "wb") as f:
    f.write(response_metadata.content)
print("Downloaded metadata.json")


In [ ]:
import requests

model_url_base = "https://teachablemachine.withgoogle.com/models/q8zdEmcC7/"

# Download the model.json file
model_json_url = model_url_base + "model.json"
response_json = requests.get(model_json_url)
with open("model.json", "wb") as f:
    f.write(response_json.content)
print("Downloaded model.json")

# Download the weights.bin file
weights_bin_url = model_url_base + "weights.bin"
response_weights = requests.get(weights_bin_url)
with open("weights.bin", "wb") as f:
    f.write(response_weights.content)
print("Downloaded weights.bin")

# Download the metadata.json file (optional, but useful)
metadata_url = model_url_base + "metadata.json"
response_metadata = requests.get(metadata_url)
with open("metadata.json", "wb") as f:
    f.write(response_metadata.content)
print("Downloaded metadata.json")


In [ ]:
import requests

model_url_base = "https://teachablemachine.withgoogle.com/models/1ujgBEAu3/"

# Download the model.json file
model_json_url = model_url_base + "model.json"
response_json = requests.get(model_json_url)
with open("model.json", "wb") as f:
    f.write(response_json.content)
print("Downloaded model.json")

# Download the weights.bin file
weights_bin_url = model_url_base + "weights.bin"
response_weights = requests.get(weights_bin_url)
with open("weights.bin", "wb") as f:
    f.write(response_weights.content)
print("Downloaded weights.bin")

# Download the metadata.json file (optional, but useful)
metadata_url = model_url_base + "metadata.json"
response_metadata = requests.get(metadata_url)
with open("metadata.json", "wb") as f:
    f.write(response_metadata.content)
print("Downloaded metadata.json")


Now that you have downloaded the models, you  can simply use this code to run the machine learning model.

In [ ]:
import tensorflow as tf
import tensorflowjs as tfjs
import numpy as np
from PIL import Image, ImageOps
import json
import os

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Define file paths
model_json_path = 'model.json'
metadata_path = 'metadata.json' # This file contains the labels

# --- Load the model ---
# Use tensorflowjs library to load the model from the JSON file
try:
    model = tfjs.converters.load_keras_model(model_json_path)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    # You might need to adjust paths if weights.bin is not in the same folder

# --- Load the labels from metadata.json ---
with open(metadata_path, 'r') as f:
    metadata = json.load(f)
    # The labels are stored in the 'labels' key within metadata.json
    class_labels = metadata.get('labels', [])

if not class_labels:
    print("Warning: Could not find labels in metadata.json.")

# The Teachable Machine model expects a specific input shape and normalization
IMAGE_SIZE = 224 # Most TM models are 224x224
data = np.ndarray(shape=(1, IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)

# Function to process an image for the model
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    # Resize the image to be at least 224x224 and then crop to 224x224
    size = (IMAGE_SIZE, IMAGE_SIZE)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

    # Turn the image into a numpy array
    image_array = np.asarray(image)

    # Normalize the image (TM uses specific normalization: (0-255) / 127.0 - 1)
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1.0

    # Load the image into the array
    data[0] = normalized_image_array

# --- Example of how to use it: ---
test_image_path = 'testing/test_image.png' #Path on github

if os.path.exists(test_image_path):
    preprocess_image(test_image_path)

    # Run the inference
    prediction = model.predict(data)

    # Get the index of the class with the highest confidence
    predicted_class_index = np.argmax(prediction)

    if predicted_class_index < len(class_labels):
        predicted_class_label = class_labels[predicted_class_index]
        confidence_score = prediction[0][predicted_class_index]

        print("-" * 40)
        print(f"Predicted class: {predicted_class_label}")
        print(f"Confidence score: {confidence_score:.2f}")
        print("-" * 40)
    else:
        print("Prediction successful, but index out of bounds for labels list.")

else:
    print(f"\nError: Test image not found at '{test_image_path}'. Please update the path.")

